In [18]:
# Clone this repository to use the utils
!git clone https://github.com/KeremTurgutlu/self_supervised.git

Cloning into 'self_supervised'...


In [19]:
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np
import random
import time
import os

from itertools import groupby
from tqdm import tqdm

tf.random.set_seed(555)
np.random.seed(555)

In [8]:
!pip install -q wandb

In [9]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tnapier. Use `wandb login --relogin` to force relogin


True

In [ ]:
from self_supervised.vision.swav import *
dls = get_dls(resize, bs)
encoder = create_encoder("xresnet34", n_in=3, pretrained=False) # a fastai encoder
#encoder = create_encoder("tf_efficientnet_b4_ns", n_in=3, pretrained=False) # a timm encoder
model = create_swav_model(encoder, hidden_size=2048, projection_size=128)
aug_pipelines = get_swav_aug_pipelines(num_crops=[2,6],
                                       crop_sizes=[128,96], 
                                       min_scales=[0.25,0.05],
                                       max_scales=[1.0,0.3])
learn = Learner(dls, model, cbs=[SWAV(aug_pipelines=aug_pipelines, crop_assgn_ids=[0,1], K=bs*2**6, queue_start_pct=0.5)])
learn.fit_flat_cos(100, 1e-2)

In [7]:
train_ds = "D:\\PhD\\data\\train"
test_ds = "D:\\PhD\\data\\val"

In [10]:
# Configs
BS = 32
SIZE_CROPS = [224, 96]
NUM_CROPS = [2, 3]
MIN_SCALE = [0.14, 0.05] 
MAX_SCALE = [1., 0.14]

In [13]:
# Experimental options
options = tf.data.Options()
options.experimental_optimization.noop_elimination = True
# options.experimental_optimization.map_vectorization.enabled = True
options.experimental_optimization.apply_default_optimizations = True
options.experimental_deterministic = False
options.experimental_threading.max_intra_op_parallelism = 1

In [14]:
# Get multiple data loaders
trainloaders = multicrop_dataset.get_multires_dataset(train_ds,
    size_crops=SIZE_CROPS,
    num_crops=NUM_CROPS,
    min_scale=MIN_SCALE,
    max_scale=MAX_SCALE,
    options=options)

AttributeError: 'str' object has no attribute 'shuffle'

In [15]:
# Prepare the final data loader

AUTO = tf.data.experimental.AUTOTUNE

# Zipping 
trainloaders_zipped = tf.data.Dataset.zip(trainloaders)

# Final trainloader
trainloaders_zipped = (
    trainloaders_zipped
    .batch(BS)
    .prefetch(AUTO)
)

im1, im2, im3, im4, im5 = next(iter(trainloaders_zipped))
print(im1.shape, im2.shape, im3.shape, im4.shape, im5.shape)

NameError: name 'trainloaders' is not defined

In [1]:
%pip install lightly

Note: you may need to restart the kernel to use updated packages.


In [2]:
import wandb

wandb.login()

wandb.init(project="a2o-swav-lightly", entity="tnapier")

wandb.config = {
  "epochs": 400,
  "batch_size": 16
}

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tnapier. Use `wandb login --relogin` to force relogin


In [3]:
# Note: The model and training settings do not follow the reference settings
# from the paper. The settings are chosen such that the example can easily be
# run on a small dataset with a single GPU.

import torch
from torch import nn
import torchvision

from lightly.data import LightlyDataset
from lightly.data import SwaVCollateFunction
from lightly.loss import SwaVLoss
from lightly.models.modules import SwaVProjectionHead
from lightly.models.modules import SwaVPrototypes


class SwaV(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.projection_head = SwaVProjectionHead(512, 512, 128)
        self.prototypes = SwaVPrototypes(128, n_prototypes=512)

    def forward(self, x):
        x = self.backbone(x).flatten(start_dim=1)
        x = self.projection_head(x)
        x = nn.functional.normalize(x, dim=1, p=2)
        p = self.prototypes(x)
        return p


resnet = torchvision.models.resnet18()
backbone = nn.Sequential(*list(resnet.children())[:-1])
model = SwaV(backbone)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# we ignore object detection annotations by setting target_transform to return 0
# pascal_voc = torchvision.datasets.VOCDetection(
#     "datasets/pascal_voc", download=True, target_transform=lambda t: 0
# )
#dataset = LightlyDataset.from_torch_dataset(pascal_voc)
# or create a dataset from a folder containing images or videos:
dataset = LightlyDataset("D:\\PhD\\data\\train")

collate_fn = SwaVCollateFunction()

dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=16,
    collate_fn=collate_fn,
    shuffle=True,
    drop_last=True,
    num_workers=8,
)

criterion = SwaVLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

print("Starting Training")
for epoch in range(400):
    total_loss = 0
    for batch, _, _ in dataloader:
        model.prototypes.normalize()
        multi_crop_features = [model(x.to(device)) for x in batch]
        high_resolution = multi_crop_features[:2]
        low_resolution = multi_crop_features[2:]
        loss = criterion(high_resolution, low_resolution)
        total_loss += loss.detach()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    avg_loss = total_loss / len(dataloader)
    print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")
    wandb.log({"avg_loss": avg_loss}) 

c:\Users\Thomas\conda\envs\audiohandling\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Starting Training
epoch: 00, loss: 6.26055
epoch: 01, loss: 6.17783
epoch: 02, loss: 6.15405
epoch: 03, loss: 6.10882
epoch: 04, loss: 6.09704
epoch: 05, loss: 6.06793
epoch: 06, loss: 6.04098
epoch: 07, loss: 6.00133
epoch: 08, loss: 6.00705
epoch: 09, loss: 5.96123
epoch: 10, loss: 5.98054
epoch: 11, loss: 5.94094
epoch: 12, loss: 5.93431
epoch: 13, loss: 5.94066
epoch: 14, loss: 5.90587
epoch: 15, loss: 5.90833
epoch: 16, loss: 5.89842
epoch: 17, loss: 5.87865
epoch: 18, loss: 5.86030
epoch: 19, loss: 5.81248
epoch: 20, loss: 5.81864
epoch: 21, loss: 5.79913
epoch: 22, loss: 5.79356
epoch: 23, loss: 5.77395
epoch: 24, loss: 5.75880
epoch: 25, loss: 5.75692
epoch: 26, loss: 5.74604
epoch: 27, loss: 5.71765
epoch: 28, loss: 5.70078
epoch: 29, loss: 5.69855
epoch: 30, loss: 5.69460
epoch: 31, loss: 5.69296
epoch: 32, loss: 5.70678
epoch: 33, loss: 5.68260
epoch: 34, loss: 5.66072
epoch: 35, loss: 5.66798
epoch: 36, loss: 5.66398
epoch: 37, loss: 5.65880
epoch: 38, loss: 5.66236
epoch: 

RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 2359296 bytes.

In [4]:
torch.save({'model': model.state_dict()}, 'my_checkpoint.pth')

In [2]:
import torch
from torch import nn
import torchvision
PATH = "my_checkpoint.pth"
model = torch.load(PATH)
model.eval()

c:\Users\Thomas\conda\envs\audiohandling\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: 'dict' object has no attribute 'eval'